<a href="https://colab.research.google.com/github/lucacamus13/FinSentiment/blob/main/finsentiment_individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 FinSentiment 2.1: Refactorización Quality-of-Life

Versión mejorada con feedback de usuario para reducir ruido legal y mejorar la visualización.

### Mejoras v2.1:
1. **Filtro Legal**: Se eliminan oraciones 'disclaimer' (Forward-Looking Statements) para purificar el sentimiento.
2. **Stopwords Extendidas**: Limpieza de palabras financieras genéricas ('may', 'company', 'results') en nubes de palabras.
3. **Score Normalizado**: Visualización de **Z-Score** (desviación vs media) para ver cambios relativos, no absolutos.

In [ ]:
# @title 1. Instalación de Dependencias
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4 yfinance

In [ ]:
# @title 2. Definición del Motor (Core Engine v2.1)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from collections import Counter
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# Configurar estilos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 7]

# --- MÓDULO 1: INGESTA ---
class SECLoader:
    def __init__(self, data_dir="data", email="research@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 2):
        path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        if os.path.exists(path):
             print(f"[>] Archivos para {ticker} ya descargados.")
             return

        print(f"[>] Descargando {amount} reportes para {ticker}...")
        try:
            self.downloader.get("10-K", ticker, limit=amount)
            print("[+] Descarga completa.")
        except Exception as e:
            print(f"[!] Error en descarga: {e}")

    def extract_date(self, content: str) -> str:
        patterns = [
            r'FILED AS OF DATE:\s+(\d{8})',
            r'CONFORMED PERIOD OF REPORT:\s+(\d{8})'
        ]
        for p in patterns:
            match = re.search(p, content)
            if match:
                date_str = match.group(1)
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return None

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        patterns = [r'Item\s+7\.\s+Management', r"Management's\s+Discussion", r'Item\s+7\.']
        start_idx = -1
        for p in patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match: start_idx = match.start(); break
        if start_idx == -1: return text[:50000]
        return text[start_idx:start_idx+30000]

    def process_filings(self, ticker: str):
        raw_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        processed_data = []
        for root, _, files in os.walk(raw_path):
            for file in files:
                if file.lower().endswith(".txt") and "primary" not in file:
                    try:
                        with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                        mda = self.extract_mda(content)
                        date = self.extract_date(content)
                        if len(mda) > 500:
                            processed_data.append({'text': mda, 'date': date, 'accession': file})
                    except: pass
        return processed_data

# --- MÓDULO 2 PREPROCESAMIENTO (CON FILTRO LEGAL) ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return "".join(ch for ch in text if ch.isprintable()).strip()

    def is_legal_noise(self, sentence: str) -> bool:
        # Palabras clave de disclaimers y riesgo legal
        legal_keywords = [
            'forward-looking', 'safe harbor', 'uncertainty', 'may differ',
            'subject to error', 'actual results', 'factors that could cause',
            'statements regarding', 'cautionary note', 'risk factors',
            'include but are not limited to', 'assumptions'
        ]
        s_lower = sentence.lower()
        return any(kw in s_lower for kw in legal_keywords)

    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        valid_sentences = []
        for s in sentences:
            s = s.strip()
            # Filtro 1: Longitud
            if len(s) > 20 and len(s.split()) >= 4:
                # Filtro 2: Ruido Legal (Falso Negativo Commmon Source)
                if not self.is_legal_noise(s):
                    valid_sentences.append(s)
        return valid_sentences

# --- MÓDULO 3: MODELO FINBERT ---
class FinBertModel:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"[*] Cargando FinBERT en {self.device}...")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert").to(self.device)
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict(self, sentences):
        if not sentences: return pd.DataFrame()
        batch_size = 32
        results = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()
            for j, s in enumerate(batch):
                idx = np.argmax(probs[j])
                results.append({
                    "sentence": s,
                    "sentiment": self.labels[idx],
                    "pos_val": probs[j][0],
                    "neg_val": probs[j][1]
                })
        return pd.DataFrame(results)

# --- MÓDULO 4: DATA Y ANALYTICS ---
def get_market_data(ticker, start_date, end_date):
    print(f"[*] Descargando precios de {ticker}...")
    df = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return df['Close']

def analyze_keywords(sentences, top_k=10):
    # Stopwords expandidas (Legal + Financiero Genérico)
    stopwords = set([
        'the', 'and', 'of', 'to', 'in', 'a', 'that', 'for', 'is', 'on', 'with', 'as',
        'our', 'we', 'are', 'by', 'it', 'from', 'an', 'be', 'files', 'company',
        # Solicitadas por el usuario:
        'may', 'such', 'period', 'results', 'year', 'quarter', 'other', 'have', 'million', 'billion'
    ])
    words = []
    for s in sentences:
        tokens = re.findall(r'\b[a-zA-Z]{3,}\b', s)
        words.extend([w.lower() for w in tokens if w.lower() not in stopwords])
    return Counter(words).most_common(top_k)

def show_dashboard(ticker, results_df, market_data=None):
    print("\n" + "="*60 + f"\n SCOREBOARD: {ticker}\n" + "="*60)

    # 1. Normalización (Z-Score)
    summary = results_df.groupby('date')[['pos_val', 'neg_val']].mean()
    summary['net_score'] = summary['pos_val'] - summary['neg_val']

    # Calcular Z-Score para ver cambios relativos
    mean_score = summary['net_score'].mean()
    std_score = summary['net_score'].std()
    if std_score == 0: std_score = 1 # Evitar Div0
    summary['z_score'] = (summary['net_score'] - mean_score) / std_score

    # Mostrar tabla coloreada por Z-Score (lo importante es si sube o baja vs media)
    print("[Métrica Clave] Z-Score: Desviación Estándar respecto a la media histórica.")
    display(summary.style.background_gradient(cmap='RdYlGn', subset=['z_score']))

    # 2. Visualización Precio vs Sentimiento Normalizado
    if market_data is not None:
        fig, ax1 = plt.subplots(figsize=(12, 6))
        color = '#2E86AB'
        ax1.set_xlabel('Fecha')
        ax1.set_ylabel('Sentiment Z-Score (Normalizado)', color=color, fontweight='bold')
        # Graficar Z-Score en lugar de raw score
        ax1.plot(pd.to_datetime(summary.index), summary['z_score'], 'o-', color=color, linewidth=2)
        ax1.axhline(0, color='gray', linestyle='--', alpha=0.5, label='Media Histórica')
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()
        color = '#F24236'
        ax2.set_ylabel(f'Precio {ticker} ($)', color=color, fontweight='bold')
        ax2.plot(market_data.index, market_data, color=color, alpha=0.6, linewidth=1.5)
        ax2.tick_params(axis='y', labelcolor=color)

        plt.title(f"{ticker}: Precio vs Sentimiento Relativo (Z-Score)", fontsize=14)
        plt.show()

    # 3. Deep Dive (Último reporte)
    latest_date = results_df['date'].max()
    latest_df = results_df[results_df['date'] == latest_date]

    print(f"\n>>> DEEP DIVE (Reporte {latest_date})")
    print("\n[TOP 5 POSITIVAS]")
    for _, r in latest_df.nlargest(5, 'pos_val').iterrows():
        print(f"(+) {r['pos_val']:.2f}: {r['sentence'][:150]}...")

    print("\n[TOP 5 NEGATIVAS - FILTRADAS]")
    for _, r in latest_df.nlargest(5, 'neg_val').iterrows():
        print(f"(-) {r['neg_val']:.2f}: {r['sentence'][:150]}...")

    pos_text = latest_df[latest_df['sentiment']=='positive']['sentence']
    neg_text = latest_df[latest_df['sentiment']=='negative']['sentence']
    print(f"\n[KEYWORDS FRECUENTES] (Sin stopwords financieras)")
    print(f"Positivas: {[k for k,v in analyze_keywords(pos_text)]}")
    print(f"Negativas: {[k for k,v in analyze_keywords(neg_text)]}")

In [ ]:
# @title 3. EJECUCIÓN MAESTRA
TICKER = "ADBE" # @param {type:"string"}
NUM_REPORTS = 6 # @param {type:"integer"}

# 1. Ingesta
loader = SECLoader()
loader.download_filings(TICKER, NUM_REPORTS)
raw_docs = loader.process_filings(TICKER)

if not raw_docs:
    print("[!] No se encontraron datos.")
else:
    # 2. Análisis Robustos
    model = FinBertModel()
    prep = TextPreprocessor()

    all_results = []
    print("\n[*] Iniciando análisis de sentimeinto...")
    for doc in raw_docs:
        # Ahora split_sentences aplica el FILTRO LEGAL automáticamente
        sentences = prep.split_sentences(prep.clean_text(doc['text']))
        print(f"Oraciones válidas en {doc.get('date')}: {len(sentences)}")

        df_sent = model.predict(sentences)
        if not df_sent.empty:
            df_sent['date'] = doc.get('date', '2023-01-01')
            all_results.append(df_sent)

    # 3. Dashboard + Market Data
    if all_results:
        final_df = pd.concat(all_results)
        dates = pd.to_datetime(final_df['date']).sort_values()
        start = dates.min() - timedelta(days=30)
        end = dates.max() + timedelta(days=30)

        try:
            prices = get_market_data(TICKER, start, end)
            show_dashboard(TICKER, final_df, prices)
        except Exception as e:
            print(f"[!] Falló descarga de precios: {e}")
            show_dashboard(TICKER, final_df)

[>] Descargando 6 reportes para ADBE...
[+] Descarga completa.
[*] Cargando FinBERT en cpu...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...


[*] Iniciando análisis de sentimeinto...
Oraciones válidas en 2022-01-21: 137
Oraciones válidas en 2024-01-17: 131
Oraciones válidas en 2020-01-21: 131
Oraciones válidas en 2021-01-15: 134
Oraciones válidas en 2023-01-17: 138
Oraciones válidas en 2025-01-13: 130
[*] Descargando precios de ADBE...


/tmp/ipython-input-3262885517.py:143: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date, progress=False)


[!] Falló descarga de precios: 'Adj Close'

 SCOREBOARD: ADBE
[Métrica Clave] Z-Score: Desviación Estándar respecto a la media histórica.


,pos_val,neg_val,net_score,z_score
date,,,,
2020-01-21,0.264601,0.018565,0.246036,-1.672424
2021-01-15,0.290984,0.019606,0.271378,-0.763882
2022-01-21,0.330370,0.017840,0.312530,0.711513
2023-01-17,0.325240,0.024077,0.301163,0.303977
2024-01-17,0.328103,0.017715,0.310388,0.634705
2025-01-13,0.335179,0.020568,0.314611,0.786117



>>> DEEP DIVE (Reporte 2025-01-13)

[TOP 5 POSITIVAS]
(+) 0.94: We have made significant investments to broaden the scale and size of all these routes to market and believe these investments will result in continue...
(+) 0.93: In our Adobe Document Cloud business, we expect to drive sustained long-term revenue growth through an expansion of our customer base by continuing to...
(+) 0.93: Further, our generative AI capabilities are increasing the value of our existing subscription products, expanding our potential customer base and incr...
(+) 0.92: We also maintain several strategic partnerships with other technology companies that allow us to increase our market reach....
(+) 0.91: By building on these features and capabilities, we increase the value we provide our customers and create a competitive differentiation in the market....

[TOP 5 NEGATIVAS - FILTRADAS]
(-) 0.63: Changes in and Disagreements with Accountants on Accounting and Financial Disclosure 93 Item 9A....
(-) 0.22: D